In [1]:
# import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCServer

In [2]:
# import SimpleXMLRPCRequestHandler
from xmlrpc.server import SimpleXMLRPCRequestHandler
import threading

In [3]:
# Batasi hanya pada path /RPC2 saja supaya tidak bisa mengakses path lainnya
class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

In [4]:
# Buat server
server = SimpleXMLRPCServer(('192.168.100.17',8008), requestHandler=RequestHandler) 

server.register_introspection_functions()

# buat data struktur dictionary untuk menampung nama_kandidat dan hasil voting
voting = {
    "namaKandidat" : [],
    "hasilVote" : []
}
# kode setelah ini adalah critical section, menambahkan vote tidak boeh terjadi race condition
# siapkan lock
lock = threading.Lock()

In [5]:
#  buat fungsi bernama vote_candidate()
def vote_candidate(x):
    # critical section dimulai harus dilock
    lock.acquire()
    # jika kandidat ada dalam dictionary maka tambahkan  nilai votenya
    if len(voting['namaKandidat']) is 0:
        voting['namaKandidat'].append(x)
        voting['hasilVote'].append(1)
    else:
        for i in range(0, len(voting['namaKandidat'])):
            if voting['namaKandidat'][i] == x:
                voting['hasilVote'][i] += 1
                break
            if (i == len(voting['namaKandidat'])-1):
                voting['namaKandidat'].append(x)
                voting['hasilVote'].append(1)

    # critical section berakhir, harus diunlock
    lock.release()
    return True

In [6]:
 # register fungsi vote_candidate() sebagai vote
server.register_function(vote_candidate, 'vote')

<function __main__.vote_candidate(x)>

In [7]:
# buat fungsi bernama querry_result
def querry_result():
    # critical section dimulai
    lock.acquire()

    # hitung total vote yang ada
    totalVote = sum(voting['hasilVote'])

    # hitung hasil persentase masing-masing kandidat
    persen = []
    for i in range(0, len(voting['namaKandidat'])):
        hasil = voting['hasilVote'][i]
        persen.append((hasil/totalVote) * 100)

    # critical section berakhir
    lock.release()
    return [
        voting['namaKandidat'],
        voting['hasilVote'],
        persen
    ]

In [8]:
 # register querry_result sebagai querry
server.register_function(querry_result, 'querry')

print ("Server voting berjalan...")

Server voting berjalan...


In [ ]:
# Jalankan server
server.serve_forever()

192.168.100.6 - - [13/Feb/2020 13:12:08] "POST /RPC2 HTTP/1.1" 200 -
192.168.100.6 - - [13/Feb/2020 13:12:08] "POST /RPC2 HTTP/1.1" 200 -
192.168.100.6 - - [13/Feb/2020 13:12:08] "POST /RPC2 HTTP/1.1" 200 -
192.168.100.6 - - [13/Feb/2020 13:12:08] "POST /RPC2 HTTP/1.1" 200 -
192.168.100.6 - - [13/Feb/2020 13:12:08] "POST /RPC2 HTTP/1.1" 200 -
